In [77]:
import os
from tensorflow.keras import models, layers, activations, optimizers, utils, losses, initializers, metrics, callbacks

In [78]:
path = './dataset'
epochs = 100
batch_size = 32
patience = 5
learning_rate = 0.001
model_path = './dataset/model.keras'

In [ ]:
model = models.Sequential([
    layers.Resizing(56, 56),
    layers.Rescaling(1.0/255),
    layers.RandomRotation((-0.2, 0.2)),

    layers.Conv2D(32, (3, 3), # 32 @ 54x54
         activation = 'relu',
         kernel_initializer = initializers.RandomNormal()
    ),
    layers.MaxPooling2D((2, 2)), # 32 @ 27x27

    layers.Conv2D(64, (3, 3), # 2048 @ 24x24
        activation = 'relu',
        kernel_initializer = initializers.RandomNormal()
    ),
    layers.MaxPooling2D((2, 2)), # 2048 @ 8x8

    layers.Flatten(), # ~125k parametros

    layers.Dropout(0.2), # 20% dos neurônios serão desativados
    layers.Dense(1024,
        activation = 'relu',
        kernel_initializer = initializers.RandomNormal()
    ),

    layers.Dense(1024,
        activation = 'relu',
        kernel_initializer = initializers.RandomNormal()
    ),
    
    layers.Dense(1024,
        activation = 'relu', #activations.relu()
        kernel_initializer = initializers.RandomNormal()
    ),

    layers.Dense(11,
        activation = 'sigmoid',
        kernel_initializer = initializers.RandomNormal()
    ),
                 
])

In [80]:
model.compile(
    optimizer = optimizers.Adam(
        learning_rate = learning_rate
    ),
    loss = losses.SparseCategoricalCrossentropy(),
    metrics = [ 
        metrics.SparseCategoricalAccuracy()
    ]
)

In [81]:
train = utils.image_dataset_from_directory(
    path,
    validation_split=0.2,
    subset = "training",
    shuffle = True,
    seed = 423,
    image_size = (224, 224),
    batch_size = batch_size
)

test = utils.image_dataset_from_directory(
    path,
    validation_split=0.2,
    subset = "validation",
    shuffle = True,
    seed = 42,
    image_size = (224, 224),
    batch_size = batch_size
)

Found 3347 files belonging to 11 classes.
Using 2678 files for training.
Found 3347 files belonging to 11 classes.
Using 669 files for validation.


In [82]:
model.fit(
    train,
    validation_data = test,
    epochs = epochs,
    callbacks = [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = 1
        ),
        callbacks.ModelCheckpoint(
            filepath = model_path,
            save_weights_only = False,
            monitor = 'loss',
            mode = 'min',
            save_best_only = True
        )
    ]
)

Epoch 1/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 15s 152ms/step - loss: 2.6317 - sparse_categorical_accuracy: 0.1603 - val_loss: 2.1434 - val_sparse_categorical_accuracy: 0.2227
Epoch 2/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 13s 156ms/step - loss: 2.1550 - sparse_categorical_accuracy: 0.2270 - val_loss: 2.0075 - val_sparse_categorical_accuracy: 0.3214
Epoch 3/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 164ms/step - loss: 2.1286 - sparse_categorical_accuracy: 0.2375 - val_loss: 2.0824 - val_sparse_categorical_accuracy: 0.2795
Epoch 4/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 13s 151ms/step - loss: 2.0060 - sparse_categorical_accuracy: 0.2887 - val_loss: 1.9386 - val_sparse_categorical_accuracy: 0.3244
Epoch 5/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 13s 153ms/step - loss: 2.0071 - sparse_categorical_accuracy: 0.2754 - val_loss: 1.9419 - val_sparse_categorical_accuracy: 0.2915
Epoch 6/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 12s 148ms/step - loss: 1.9154 - sparse_categorical_accuracy: 0.3095 - val_loss: 1.7619 - val_sparse_categorical_accuracy: